In [2]:
from api_endpoints import api_teams

In [3]:
teams = api_teams()
teams_id = []
for team in teams["teams"]:
    teams_id.append(team["id"])

In [4]:
print(len(teams_id), len(teams_id[0]))

535 36


In [5]:
from api_endpoints import api_team_roasters

First, fill the parent tables of team
1. from team roasters ep  
    1. conferences
    2. divisions
    3. venues
2. then teams

In [ ]:
roaster_info = api_team_roasters(teams_id[0])


In [9]:
roaster_info

{'id': '30abfee7-57a7-4984-9a1c-55cb28970da3',
 'name': '49ers',
 'market': 'Charlotte',
 'alias': 'CHAR',
 'founded': 1946,
 'mascot': 'Norm the Niner',
 'fight_song': 'Charlotte 49ers Fight Song',
 'championships_won': 0,
 'conference_titles': 0,
 'playoff_appearances': 0,
 'franchise': {'id': '30abfee7-57a7-4984-9a1c-55cb28970da3', 'name': '49ers'},
 'venue': {'id': '5189127d-ea58-4917-84b8-8456f3abb8b2',
  'name': 'Jerry Richardson Stadium',
  'city': 'Charlotte',
  'state': 'NC',
  'country': 'USA',
  'zip': '28262',
  'address': '9201 University City Boulevard',
  'capacity': 15314,
  'surface': 'artificial',
  'roof_type': 'outdoor',
  'location': {'lat': '35.3105', 'lng': '-80.7401'}},
 'division': {'id': '43d3cede-6b8f-4158-bc7f-c3c7365e5b32',
  'name': 'I-A',
  'alias': 'FBS'},
 'conference': {'id': '49602742-5d66-412e-8a58-ab1e613d593c',
  'name': 'American',
  'alias': 'AAC'},
 'coaches': [{'id': '35ccf611-95e3-40b0-8de1-986a72aa5020',
   'full_name': 'Tim Albin',
   'first

In [10]:
print(type(roaster_info["founded"]))
print(type(roaster_info["championships_won"]))
print(type(roaster_info["venue"]['zip']))

<class 'int'>
<class 'int'>
<class 'str'>


In [18]:
import pandas as pd

In [54]:
venue_df = pd.DataFrame(columns=["venue_id", "name","city", "state", "country", "zip", "address", "capacity", "surface", "roof_type", "latitude", "longitude"])
conferene_df = pd.DataFrame(columns=["conference_id", "name", "alias"])
divison_df = pd.DataFrame(columns=["division_id", "name", "alias"])
teams_df = pd.DataFrame(columns= ['team_id', 'market', 'name', 'alias', 'founded', 'mascot', 'fight_song', 'championships_won', 'conference_id', 'division_id', 'venue_id'])
players_df = pd.DataFrame(columns=['player_id', 'first_name', 'last_name', 'abbr_name', 'birth_place', 'position', 'height', 'weight', 'status', 'eligibility', 'team_id'])
coaches_df = pd.DataFrame(columns=["coach_id", "full_name", "position", "team_id"])

In [44]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

def _fetch_roaster(tid):
    try:
        return api_team_roasters(tid)
    except Exception as e:
        return {"id": tid, "_error": str(e)}

max_workers = min(50, (os.cpu_count() or 1) * 5, len(teams_id))
roasters_list = []
with ThreadPoolExecutor(max_workers=max_workers) as ex:
    futures = [ex.submit(_fetch_roaster, tid) for tid in teams_id]
    for i, fut in enumerate(as_completed(futures), 1):
        roasters_list.append(fut.result())
        if i % 50 == 0 or i == len(teams_id):
            print(f"Fetched {i}/{len(teams_id)}")

Fetched 50/535
Fetched 100/535
Fetched 150/535
Fetched 200/535
Fetched 250/535
Fetched 300/535
Fetched 350/535
Fetched 400/535
Fetched 450/535
Fetched 500/535
Fetched 535/535


In [49]:
roasters_list[0]

{'id': '5fb09809-05fa-42cf-8636-311b8b4c2c04',
 'name': 'Bears',
 'market': 'Pikeville',
 'alias': 'UOP',
 'franchise': {'id': '5fb09809-05fa-42cf-8636-311b8b4c2c04', 'name': 'Bears'},
 'division': {'id': 'a590be2a-ad78-4ca1-818b-a14b4c369100',
  'name': 'NAIA',
  'alias': 'NAIA'},
 'conference': {'id': 'eb7f8197-a82d-4ab3-831b-5a2c647618e0',
  'name': 'Mid-South Conference',
  'alias': 'MID-SOUTH'},
 'coaches': [],
 'players': [],
 '_comment': 'Generation started @ 2025-10-22 17:03:39 +0000 ended @ 2025-10-22 17:03:39 +0000'}

In [55]:
def _get(d, keys, default=None):
    """Safe nested dict getter: _get(d, ['a','b']) -> d['a']['b'] or default."""
    cur = d
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur

for roaster in roasters_list:
    # skip non-roaster results (e.g. error messages)
    if not isinstance(roaster, dict) or 'id' not in roaster:
        continue

    venue_dict = {
        'venue_id': _get(roaster, ['venue', 'id']),
        'name': _get(roaster, ['venue', 'name']),
        'city': _get(roaster, ['venue', 'city']),
        'state': _get(roaster, ['venue', 'state']),
        'country': _get(roaster, ['venue', 'country']),
        'zip': _get(roaster, ['venue', 'zip']),
        'address': _get(roaster, ['venue', 'address']),
        'capacity': _get(roaster, ['venue', 'capacity']),
        'surface': _get(roaster, ['venue', 'surface']),
        'roof_type': _get(roaster, ['venue', 'roof_type']),
        'latitude': _get(roaster, ['venue', 'location', 'lat']),
        'longitude': _get(roaster, ['venue', 'location', 'lng']),
    }

    conference_dict = {
        'conference_id': _get(roaster, ['conference', 'id']),
        'name': _get(roaster, ['conference', 'name']),
        'alias': _get(roaster, ['conference', 'alias']),
    }

    division_dict = {
        'division_id': _get(roaster, ['division', 'id']),
        'name': _get(roaster, ['division', 'name']),
        'alias': _get(roaster, ['division', 'alias']),
    }

    teams_dict = {
        'team_id': _get(roaster, ['id']),
        'market': _get(roaster, ['market']),
        'name': _get(roaster, ['name']),
        'alias': _get(roaster, ['alias']),
        'founded': _get(roaster, ['founded']),
        'mascot': _get(roaster, ['mascot']),
        'fight_song': _get(roaster, ['fight_song']),
        'championships_won': _get(roaster, ['championships_won']),
        'conference_id': _get(roaster, ['conference', 'id']),
        'division_id': _get(roaster, ['division', 'id']),
        'venue_id': _get(roaster, ['venue', 'id']),
    }

    # Insert only when id is present and not already in target DF
    if venue_dict['venue_id'] is not None and venue_dict['venue_id'] not in venue_df['venue_id'].values:
        venue_df.loc[len(venue_df)] = venue_dict

    if conference_dict['conference_id'] is not None and conference_dict['conference_id'] not in conferene_df['conference_id'].values:
        conferene_df.loc[len(conferene_df)] = conference_dict

    if division_dict['division_id'] is not None and division_dict['division_id'] not in divison_df['division_id'].values:
        divison_df.loc[len(divison_df)] = division_dict

    if teams_dict['team_id'] is not None and teams_dict['team_id'] not in teams_df['team_id'].values:
        teams_df.loc[len(teams_df)] = teams_dict

    for coach in roaster.get('coaches', []) or []:
        c = {
            'coach_id': _get(coach, ['id']),
            'full_name': _get(coach, ['full_name']),
            'position': _get(coach, ['position']),
            'team_id': teams_dict['team_id'],
        }
        if c['coach_id'] is not None and c['coach_id'] not in coaches_df['coach_id'].values:
            coaches_df.loc[len(coaches_df)] = c

    for player in roaster.get('players', []) or []:
        p = {
            'player_id': _get(player, ['id']),
            'first_name': _get(player, ['first_name']),
            'last_name': _get(player, ['last_name']),
            'abbr_name': _get(player, ['abbr_name']),
            'birth_place': _get(player, ['birth_place']),
            'position': _get(player, ['position']),
            'height': _get(player, ['height']),
            'weight': _get(player, ['weight']),
            'status': _get(player, ['status']),
            'eligibility': _get(player, ['eligibility']),
            'team_id': teams_dict['team_id'],
        }
        if p['player_id'] is not None and p['player_id'] not in players_df['player_id'].values:
            players_df.loc[len(players_df)] = p


/tmp/ipykernel_303205/1783081211.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  players_df.loc[len(players_df)] = p
/tmp/ipykernel_303205/1783081211.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  players_df.loc[len(players_df)] = p
/tmp/ipykernel_303205/1783081211.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retai

In [56]:
venue_df

,venue_id,name,city,state,country,zip,address,capacity,surface,roof_type,latitude,longitude
0,eeee7a86-39bc-4f36-bdfe-f1cf2d45d695,Christy Mathewson-Memorial Stadium,Lewisburg,PA,USA,17837,Christy Mathewson Drive,13100,artificial,outdoor,40.9518,-76.8854
1,889f7722-95bd-413a-9b1b-bbec46e490c7,Kyle Field,College Station,TX,USA,77843,756 Houston Street,102733,turf,outdoor,30.6101,-96.3401
2,bda80157-308a-4156-9eb1-b1c6d204292f,Brown Field,Valparaiso,IN,USA,46383,1094 Laporte Avenue,5000,artificial,outdoor,41.4656,-87.0479
3,170baa73-abb9-4247-8aea-190c51233841,Aggie Memorial Stadium,Las Cruces,NM,USA,88003,1810 E University Avenue,30343,artificial,outdoor,32.2795,-106.7417
4,5189127d-ea58-4917-84b8-8456f3abb8b2,Jerry Richardson Stadium,Charlotte,NC,USA,28262,9201 University City Boulevard,15314,artificial,outdoor,35.3105,-80.7401
5,b6893998-4d68-45d8-b7df-da763712f785,Five Star Stadium,Macon,GA,USA,31201,1327 Stadium Drive,10200,artificial,outdoor,32.8268,-83.6529
6,381d6495-10ff-403b-9838-64e495422fee,Houck Field,Cape Girardeau,MO,USA,63701,920 Broadway Street,11015,artificial,outdoor,37.3094,-89.5311
7,0f0b1691-1f01-41f4-a34d-4f34ac0dc413,Alamodome,San Antonio,TX,USA,78203,100 Montana Street,65000,artificial,dome,29.4170,-98.4788
8,d1560e06-bfbc-4a52-94dd-253949d52cee,Jahna Field,Winter Haven,FL,USA,33880,5001 FL-540,400,turf,outdoor,28.0120,-81.8000
9,43db741f-27bd-4ddd-95de-012f663a36bc,Glass Bowl,Toledo,OH,USA,43606,3044 North Glass Bowl Drive,26038,artificial,outdoor,41.6573,-83.6136


In [57]:
divison_df

,division_id,name,alias
0,a590be2a-ad78-4ca1-818b-a14b4c369100,NAIA,NAIA
1,6457fab7-b7f1-4f6b-84be-cd713e36070d,I-AA,FCS
2,43d3cede-6b8f-4158-bc7f-c3c7365e5b32,I-A,FBS
3,0c37e018-9a60-44dd-88af-e28ce4a4184b,Division II,D2
4,5f7c0a15-2b79-4ca4-b320-136d9495d1f7,Division III,D3
5,aa5db6ca-f9b2-4ab9-8e07-6a249ddf0c82,USCAA,USCAA


In [58]:
conferene_df

,conference_id,name,alias
0,eb7f8197-a82d-4ab3-831b-5a2c647618e0,Mid-South Conference,MID-SOUTH
1,034e3dea-6190-45d6-8dc6-ca52ba2d65c0,Patriot League,PATRIOT-LEAGUE
2,c7a491c8-d0f2-4ca1-9403-f82db4452dd1,Southeastern,SEC
3,c320231c-eec0-4b9c-b09e-016280763cf2,Pioneer,PIONEER
4,359cd6eb-de34-4ae2-90a2-6e80e4d4ff62,Conference USA,CONFERENCE-USA
5,eecdc945-d69b-489a-99e4-a2d2bb16bb24,Great American Conference,GAC
6,9a47496a-d965-4a65-b2cc-b5e46081af56,Liberty League,Liberty
7,49602742-5d66-412e-8a58-ab1e613d593c,American,AAC
8,a15aa3a4-6fc3-4a25-be98-d0059836a060,Southern,SOUTHERN
9,84a842c1-e82f-11ed-b016-0fa2d9c64913,Big South-OVC,BIG-SOUTH-OVC


In [59]:
coaches_df

,coach_id,full_name,position,team_id
0,91a44aab-cc88-4eff-8058-10681070f698,Dave Cecchini,Head Coach,3522e1d5-7dc6-4586-a14f-276f56e0c60e
1,42c7f7e0-5a8e-11ec-94c4-913b0a0e0ad8,Mike Elko,Head Coach,d5199a36-7f26-4c3b-be48-9599039e232c
2,bfa672a0-bd73-11ef-989a-c9b4cf8b62d8,Andy Waddle,Head Coach,59d0852a-b283-4829-af63-60553fbafdeb
3,92672703-eb7d-4c09-9220-99ecb1217753,Tony Sanchez,Head Coach,f6f4a672-6368-45aa-98d0-3756da844114
4,35ccf611-95e3-40b0-8de1-986a72aa5020,Tim Albin,Head Coach,30abfee7-57a7-4984-9a1c-55cb28970da3
5,56445fb0-b604-11ee-a4ff-07fd0755a39b,Mike Jacobs,Head Coach,cc1b918c-1523-4412-8b1e-2219d88f11b8
6,c055810a-383b-4844-91af-0f17e56bf31c,Tom Matukewicz,Head Coach,4eb165ff-ebe8-4a9e-8a57-34ec90f112e4
7,093855e6-a27c-4e5f-992c-fe704ec9e4f7,Jeff Traylor,Head Coach,b845a41f-2606-478f-a458-3fee79588ad8
8,54dbd7c7-c776-48d0-ba98-404ecb5068c0,Darrin Chiaverini,Head Coach,72566033-9632-432f-b63a-3942e54bd7c5
9,80d0eb70-8562-11f0-ae59-6d7d8b3605e6,Diaelleo Burks,Head Coach,2ed1462d-6670-4094-a86d-d723ae3b1886


In [63]:
print(players_df.head())
# print(players_df.tail())

                              player_id first_name last_name   abbr_name  \
0  0096ac20-1cef-11ee-b92c-75a1a63d7aef      David  LoGrasso  D.LoGrasso   
1  01bf7e30-5d92-11ef-9ce4-991abb7e5524     Carter    Enyard    C.Enyard   
2  06fc5c30-c74b-11ed-9fa8-892eecf2a8d4        Sam  Milligan  S.Milligan   
3  0b4feee0-3d2b-11ee-8590-a5df520e0ca3       Dara   Adeyemi   D.Adeyemi   
4  0c358418-376b-468f-b5be-50fc6a198063      Ralph    Rucker    R.Rucker   

           birth_place position height  weight status eligibility  \
0   Lynnfield, MA, USA       DL     74   315.0    ACT          JR   
1   St. Louis, MO, USA       LB     74   230.0    ACT          SO   
2  Swarthmore, PA, USA       WR     75   220.0    ACT          JR   
3          Luton,, GBR       DL     79   280.0    ACT          JR   
4       Lucas, TX, USA       QB     72   210.0    ACT          SR   

                                team_id  
0  3522e1d5-7dc6-4586-a14f-276f56e0c60e  
1  3522e1d5-7dc6-4586-a14f-276f56e0c60e  
2 